In [219]:
from PIL import Image, ImageDraw, ImageColor
import numpy as np
import math
import itertools as it
from iteration_utilities import deepflatten

In [220]:
#color maps
def color_map_black_and_white(n):
    """A black a white color spectrum"""
    if n >= 2:
        color_step_100 = 100/(n - 1)
    else:
        color_step_100 = 0
    black_and_white = [None] * n
    for i in range(n):
        progress = int(i*color_step_100)
        black_and_white[i] = f"hsl(0,0%,{progress}%)"
    return black_and_white
    
def color_map_rainbow(n):
    """produces a rainbow ordering"""
    l = [None] * n
    hue_step = 360/n
    for i in range(n):
        hue_value = hue_step * i
        l[i] = f"hsl({hue_value},100%,50%)"
    return l

def color_map_nmp(n,m,p):
    """produces an color ordering by n varies the hue, m varies the lightness, p varies the saturation
    lexicographically, you probably want n * m * p = group_order
    """
    l = [None] * (n * m * p)
    hue_offset = 15
    hue_step = 360/n
    lightness_initial = 40
    lightness_final = 70
    lightness_step = (lightness_final - lightness_initial)/(m - 1)
    saturation_initial = 40
    saturation_final = 100
    saturation_step = (saturation_final - saturation_initial)/(p - 1)
    for i in range(n):
        hue_value = int(hue_offset + hue_step * i)
        for j in range(m):
            lightness_value = int(lightness_initial + lightness_step * j)
            for k in range(p):
                saturation_value = int(saturation_initial + saturation_step * k)
                #this code can be varied to chance which one is inside (in the ordering)
                #hue on the outside seems best
                index = i * m * p + j * p + k
                l[index] = f"hsl({hue_value},{saturation_value}%,{lightness_value}%)"
    return l

def color_map_n1n2mp(n1,n2,m,p):
    """produces an color ordering by n1 and n2 varies the hue (lexicographically with eachother),
    m varies the lightness, p varies the saturation
    lexicographically, you probably want n1 * n2 * m * p = group_order
    """
    l = [None] * (n1 * n2 * m * p)
    hue_offset = 60
    hue_step1 = 360/n1
    squeeze_factor = 1.2
    hue_step2 = hue_step1/(n2 * squeeze_factor)
    lightness_initial = 40
    lightness_final = 70
    lightness_step = (lightness_final - lightness_initial)/(m - 1)
    saturation_initial = 60
    saturation_final = 100
    saturation_step = (saturation_final - saturation_initial)/(p - 1)
    for i1 in range(n1):
        for i2 in range(n2):
            hue_value = int(hue_offset + hue_step1 * i1 + hue_step2 * i2) % 360
            for j in range(m):
                lightness_value = int(lightness_initial + lightness_step * j)
                for k in range(p):
                    saturation_value = int(saturation_initial + saturation_step * k)
                    #this code can be varied to chance which one is inside (in the ordering)
                    #hue on the outside seems best
                    index = (i1 * n2 + i2) * m * p + j + k * m
                    l[index] = f"hsl({hue_value},{saturation_value}%,{lightness_value}%)"
    return l


In [329]:
class group_elt:
    def __init__(self):
        #in general this won't be used but for this for the geometric norm of elements (w/r/t generators)
        self.norm = None
    
    @classmethod
    def e(cls):
        pass
    
    def __eq__(self,other):
        pass
    
    def __ne__(self,other):
        return not(self == other)
        
    def __mul__(self, other):
        pass
    
    def invert(self):
        pass
    
    def __xor__(self,n):
        """computes self * ... (n) ... * self. If n is negative it inverts it"""
        if n > 0:
            return self * (self ^ (n-1))
        elif n == 0:
            return self.e()
        elif n < 0:
            return self.invert() ^ (-n)
    
    def order(self):
        self_to_the_n = self
        o = 1
        while self_to_the_n != self.e():
            self_to_the_n = self_to_the_n * self
            o += 1
        return o
    
    #depricated
    def cosets(self,H):
        """suppose H is a list of group elements (hopefully a subgroup),
        then left cosets returns flatten[H, self*H,self^2 * H , ...]
        (self should have finite order)
        """
        g = self
        generate_g_times_H = []
        for i in range(g.order()):
            generate_g_times_H.append([(g ^ i) * h for h in H])
        return list(deepflatten(generate_g_times_H))

In [330]:
class permutation(group_elt):
    """Bijections from {1,2,...,n} -> {1,2, ... ,n}"""
    def __init__(self,t):
        """t is a tuple with n elements 
        which is the one line notation for the permutation
        """
        self.one_line = t
        super(permutation, self).__init__()
        
    @classmethod
    def e_n(cls,n):
        """Gives the identity on n elements"""
        l = range(1,n+1)
        return permutation(tuple(l))
    
    @classmethod
    def from_string(cls,s):
        string = str(s)
        n = len(string)
        l = [int(string[i]) for i in range(n)]
        return cls(tuple(l))
        
    def length(self):
        return len(self.one_line)
    
    def __eq__(self,other):
        if self.length() == other.length():
            return self.one_line == other.one_line
        else:
            n = max(self.length(),other.length())
            sigma = self.embed(n)
            tau = other.embed(n)
            return sigma == tau
        
    
    def __hash__(self):
        return hash(self.one_line)
    
    def __mul__(self, other):
        """multiplies with self \circ other self.length() == other.length() better be true"""
        if self.length() == other.length():
            tau = other.one_line
            product_as_list = [tau[i - 1] for i in self.one_line]
            return permutation(tuple(product_as_list))
        else:
            #print("Warning: Multiplying permutations of uneven length")
            n = max(self.length(),other.length())
            sigma = self.embed(n)
            tau = other.embed(n)
            return sigma * tau
    
    def invert(self):
        l = list(self.one_line)
        m = [l.index(i) + 1 
             for i in range(1,self.length() + 1)]
        return permutation(tuple(m))
    
    @classmethod
    def e(cls):
        """Independent Identity: Uses that I can multiply with uneven length"""
        return cls.e_n(1)
    
    def e(self):
        """Depenedent Identity: Analyzes how big self is an choose the right length identity"""
        n = self.length()
        return self.e_n(n)
    
    def __str__(self):
        if self.length() <= 9:
            return "".join(map(str,self.one_line))
        else:
            return "<" + ",".join(map(str,self.one_line)) + ">"
    
    def __repr__(self):
        return self.__str__()
    
    def embed(self,m):
        n = self.length()
        if not n <= m:
            raise Exception("Permutations only embed if you increase they're size")
        return permutation(self.one_line + tuple(range(n + 1, m + 1)))
    
    def __add__(self,other):
        """Returns the disjoint (block) sum of two permutations"""
        t = self.one_line
        n = self.length()
        u = other.one_line
        shifted_u = tuple(map(lambda x: x + n, u))
        return permutation(t + shifted_u)
    
    @classmethod
    def cycle(cls,n):
        """Gives a cycle of length n"""
        l = list(range(2,n+1))
        l.append(1)
        return permutation(tuple(l))
    
    def act(self, i):
        """Has self act on the set {1,2,..., n} in the canonical way"""
        return self.one_line[i - 1]

#For convenience
st = permutation.from_string

In [338]:
class group:
    """An ordered group"""
    def __init__(self,l):
        """l should be a list of group elements, the multiplication etc is internal"""
        self.elements = l
        
    def verify(self):
        """Verifies that self is a group (has all the laws), self should have group_elts which are hashable"""
        elts = self.elements
        e = elts[0].e()
        if len(set(elts)) != len(elts):
            raise Exception("Group has duplicates")
        
        for g in elts:
            for h in elts:
                if not (g * h in elts):
                    raise Exception("Group not closed under multiplication")
        
        for g in elts:
            if not g * e == g:
                raise Exception("The right identity law failed")
            if not e * g == g:
                raise Exception("The left identity law failed")
            if not g * g.invert() == e:
                raise Exception("The right inverse law failed")
            if not g.invert() * g == e:
                raise Exception("The right inverse law failed")
                
        for g in elts:
            for h in elts:
                for k in elts:
                    if not g * (h * k) == (g * h) * k:
                        raise Exception("The associative law failed")
        return "Passed"
    
    def order(self):
        return len(self.elements)
        
    def __str__(self):
        return str(self.elements)
    
    def left_coset(self, g):
        """Returns a list of elements of the left coset g * self"""
        elts = self.elements
        return [g * h for h in elts]
    
    def cyclic_subgroup(g):
        """Returns the subgroup generated by self 
        (self should have finite order)
        """
        return cls([g^i for i in range(g.order())])
    
    @classmethod
    def generate(cls, *args):
        """Returns the subgroup generated by args, the order comes from growing balls at 0 in the Cayley graph with generators as args"""
        #we start at r = 0 and consider the open balls (in the geometry on our target group)
        r = 0
        B_r = []
        e = args[0].e()
        e.norm = 0
        S_r = [e] #The sphere of radius r
        S_pm_duplicates = list(args) + [g.invert() for g in args]
        S_pm = []
        for g in S_pm_duplicates:
            if g not in S_pm:
                S_pm.append(g)

        while S_r:
            if debug:
                print("r =", r, "#B_r =", len(B_r), "#S_r =", len(S_r))
            r += 1
            B_r = B_r + S_r
            #the tubular radius around S_r
            N_1_S_r = [s * g for g in S_r for s in S_pm]
            S_r = []
            for g in N_1_S_r:
                if g not in B_r and g not in S_r:
                    g.norm = r
                    S_r.append(g)
        if debug:
                print("r =", r, "#B_r =", len(B_r), "#S_r =", len(S_r))
        return cls(B_r)
    
    def maxnorm(self):
        """Returns the maximum norm of elements in G"""
        if 
        norms = [g.norm for g in self.elements]
        return max(norms)
    
    def reorder_by_subgroup(self,H):
        """Takes a subgroup H of self and reorder self by H is the first elts
        this method requires * to be defined correctly
        only returns a list of the elements"""
        G = self
        H_elts = H.elements
        new_order = H_elts[:]
        remaining_elts = G.elements[:]
        try:
            remaining_elts[0] * new_order[0]
        except:
            raise Exception("H is not a subgroup of self")
        for h in H_elts:
            remaining_elts.remove(h)
            
        while remaining_elts:
            g = remaining_elts[0]
            for i in range(1,g.order()):
                new_order = new_order + H.left_coset(g ^ i)
            for k in new_order:
                try:
                    remaining_elts.remove(k)
                except ValueError:
                    pass
        return self.__class__(new_order)
        
        
    def cayley_table(self,approximate_size = 500,color_map=None,color_key=None):
        """Draws the Cayley Table of self, color map is a list of color strings for pillow to use"""
        #set up
        elts = self.elements
        order = self.order()
        
        #choose progress bar and approximate size based on size
        if order >= 200:
            print("This is going to take a while stand by:")
            progress_bar = True
            if order <= 1000:
                progress_bar_levels = 5
                if approximate_size == 500:
                    approximate_size = 1000
            elif order <= 2000:
                progress_bar_levels = 2
                if approximate_size == 500:
                    approximate_size = 2000
            else:
                progress_bar_levels = 1
            progress_bar_step = int(order/(100 / progress_bar_levels))
        else:
            progress_bar = False
        length = int(approximate_size/order) * order
        
        #choose color map
        if color_map == None:
            color_map = color_map_black_and_white(order)
            color_map.reverse()
        if color_key == 'Rainbow':
            color_map = color_map_rainbow(order)
        
        #set up draw engine
        step_count = order
        image = Image.new(mode = 'RGB', size=(length, length), color=255)
        draw = ImageDraw.Draw(image)
        step_size = int(image.width / step_count)
        if step_size == 0:
            raise Exception("Your image size is too small to draw a group this big")
        
        scaled_axis = range(0, image.width, step_size)

        #Draw the Table (abstractly)
        abstract_table = [[None] * order for i in range(order)]
        for i in range(order):
            for j in range(order):
                #I give the convention the it's the y-axis elements times the x-axis elements
                abstract_table[i][j] = elts.index(elts[j] * elts[i])
            if progress_bar:
                if i % progress_bar_step == 0:
                    print(int(i/order * 100),"% done")
        
        #Draw the table (actually)
        if progress_bar:
            print("Drawing abstract table to an image")
        for i in range(order):
            for j in range(order):
                x = scaled_axis[i]
                y = scaled_axis[j]
                rectangle = ((x,y),(x+step_size,y+step_size))
                #I give the convention the it's the y-axis elements times the x-axis elements
                c = color_map[abstract_table[i][j]]
                draw.rectangle(rectangle, fill=c)

        del draw
        image.show()

In [339]:
class sgroup(group):
    """A group embedded in S_n for some n"""
    def __init__(self,l):
        """l should be a list of permutations which has group structure"""
        super(sgroup, self).__init__(l)
        
    def __mul__(self,other):
        """Computes the direct product of self and other"""
        out_list = [g + h for (g,h) in it.product(self.elements,other.elements)]
        return sgroup(out_list)
    
    def __xor__(self,n):
        """computes self * ... (n) ... * self."""
        if n > 0:
            return self * (self ^ (n-1))
        elif n == 0:
            return self.trivial()
    
    @classmethod
    def trivial(cls):
        return cls([permutation.e_n(1)])
    
    @classmethod
    def zmodn(cls,n):
        """The cyclic group of order n"""
        sn = permutation.cycle(n)
        return cls(sn.cosets([permutation.e_n(n)]))
    
    @classmethod
    def sn_lex(cls,n):
        """The symmetric group acting on sets of size n, so of order n!
        Given in the lexicographic order
        """
        
    
    @classmethod
    def an(cls,n):
        sn_elts = cls.sn(n,'Lexicographic').elements
        an_elts = []
        standard_basis = np.identity(n)
        for sigma in sn_elts:
            matrix_rep = [standard_basis[sigma.act(i) - 1] for i in range(1,n+1)]
            if round(np.linalg.det(matrix_rep)) == 1:
                an_elts.append(sigma)
        return cls(an_elts)
    
    @classmethod
    def sn(cls,n,key = 'A_n'):
        """The symmetric group acting on sets of size n, so of order n!"""
        if key == 'Lexicographic':
            sn_elts_tuples = it.permutations(
            tuple(range(1,n+1)))
            sn_elts = [permutation(t) for t in sn_elts_tuples]
            return cls(sn_elts)
        if key == 'A_n':
            return cls.sn(n,'Lexicographic').reorder_by_subgroup(cls.an(n))
        if key == 'Geometric':
            return cls.generate(permutation.cycle(n),st(21))
        

In [357]:
debug = False
G = sgroup.sn(6, 'Geometric')
g1 = st(21345678)
g2 = st(34125678)
g3 = st(56781234)
G = sgroup.generate(g1,g2,g3)
black_and_white = color_map_black_and_white(2)
norm_colors = color_map_rainbow(G.maxnorm() + 1)
for i in range(0,G.maxnorm() + 1):
    if i % 2 == 1:
        norm_colors[i] = black_and_white[0]
colors = [norm_colors[g.norm] for g in G.elements]
G.cayley_table(color_map = colors)

In [164]:
perm4 = st
#Making an ordering for the composition series 1 < Z/2 < V_4 < A_4 < S_4
z2 = [permutation.e_n(4),perm4(2143)]
h1 = perm4(3412)
v4 = h1.cosets(z2)
h2 = perm4(2314)
a4 = h2.cosets(v4)
h3 = perm4(2134)
composition_order = sgroup(h3.cosets(a4))
composition_order.cayley_table()
#Making a 1 < Z/4 < D8 < S4 order note D8 is not normal in S4 so this will be a knit product:
#this is now broken I need a new way of doing cosets
z4 = sgroup.cyclic_subgroup(st(2341))
h1 = perm4(3214)
d8 = h1.cosets(z4)
h2 = perm4(2314)
d8_order = sgroup(h2.cosets(d8))
d8_order.cayley_table()
#1 < Z/3 = A3 < A4 < S4 order A3 is not normal in A4 and we construct A3 knit V4 = A4
z3 = perm4(2314).cyclic_subgroup()
h1 = perm4(2143)
h2 = perm4(3412)
z3h1 = h1.cosets(z3)
a4 = h2.cosets(z3h1)
knitted_a4_order = h3.cosets(a4)

In [165]:
perm5 = st
perm8 = st
#S5 composition order 1 < Z/2 < V_4 < A_4 < A_5 < S_5
g1 = perm5(21435)
g2 = perm5(34125)
g3 = perm5(23145)
g4 = perm5(23451)
g5 = perm5(21345)
z2 = g1.cosets([permutation.e_n(5)])
v4 = g2.cosets(z2)
a4 = g3.cosets(v4)
a5 = g4.cosets(a4)
s5_composition_order = g5.cosets(a5)
sgroup(s5_composition_order).cayley_table()
sgroup.sn(5).cayley_table()
#2-Sylow in S8
g1 = perm8(21345678)
g2 = perm8(34125678)
g3 = perm8(56781234)
z2 = g1.cosets([permutation.e_n(8)])
h2 = g2*g1*g2.invert()
z2wrz2 = g2.cosets(h2.cosets(z2))
h2s = [g1,h2,g2]
h3s = [g3*g*g3.invert() for g in h2s] + [g3]
z2wrz2wrz2 = h3s[3].cosets(
    h3s[2].cosets(
        h3s[1].cosets(
            h3s[0].cosets(z2wrz2)
        )
    )
)
sgroup(z2wrz2wrz2).cayley_table()